### Generate training data 

In [39]:
%%bash 
cd ~/SageMaker 
git clone https://github.com/catwhiskers/ocr_data_generator.git
cd ocr_data_generator
pip install -r requirements.txt 
    
    

  Using cached numpy-1.16.4-cp36-cp36m-manylinux1_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.1
    Uninstalling numpy-1.18.1:
      Successfully uninstalled numpy-1.18.1


fatal: destination path 'ocr_data_generator' already exists and is not an empty directory.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


#### generate training data 
* output folder 
* image folder (under output folder)
* strings to be generated 
* font directory (where ttf resides)

In [40]:
%%bash
cd ~/SageMaker/ocr_data_generator
python generate_data.py seto_font train data/ZH_1.txt setofont 
mkdir -p input 
mkdir -p input/data
mv seto_font ~/SageMaker/input/data/setofont_train

Missing modules for handwritten text generation.
['國立臺灣大學醫學院附設醫院', '三軍總醫院', '臺北榮民總醫院', '國泰綜合醫院', '臺北及林口長庚醫院', '馬偕紀念醫院', '新光吳火獅紀念醫院', '臺北市立萬芳醫院', '亞東紀念醫院', '臺中榮民總醫院', '中山醫學大學附設醫院', '中國醫藥大學附設醫院', '彰化基督教醫療財團法人彰化基督教醫院', '國立成功大學醫學院附設醫院', '奇美醫院', '高雄榮民總醫院', '高雄長庚紀念醫院', '高雄醫學大學附設中和紀念醫院', '佛教慈濟醫療財團法人花蓮慈濟醫院', '基督復臨安息日會醫療財團法人臺安醫院', '天主教耕莘醫療財團法人耕莘醫院', '敏盛綜合醫院經國總院', '馬偕紀念醫院新竹分院', '國泰綜合醫院新竹分院', '為恭紀念醫院', '衛生福利部豐原醫院', '童綜合醫院', '佛教慈濟綜合醫院臺中分院', '天主教聖馬爾定醫院', '嘉義長庚紀念醫院', '臺南市立醫院', '高雄市立大同醫院', '阮綜合醫院', '馬偕紀念醫院臺東分院', '高雄市立鳳山醫院', '三軍總醫院松山分院附設民眾診療服務處', '臺北醫學大學附設醫院', '臺北市立聯合醫院', '振興醫療財團法人振興醫院', '基督復臨安息日會醫療財團法人臺安醫院', '中心診所醫療財團法人中心綜合醫院', '醫療財團法人辜公亮基金會和信治癌中心醫院', '行天宮醫療志業醫療財團法人恩主公醫院', '天主教耕莘醫療財團法人耕莘醫院', '衛生福利部臺北醫院', '衛生福利部雙和醫院', '新北市立聯合醫院', '沙爾德聖保祿修女會醫療財團法人聖保祿醫院', '臺北榮民總醫院桃園分院', '衛生福利部桃園醫院', '國軍桃園總醫院附設民眾診療服務處', '敏盛綜合醫院', '聯新國際醫院', '國立臺灣大學醫學院附設醫院新竹分院', '馬偕紀念醫院新竹分院', '國泰醫院新竹分院', '南門綜合醫院', '國軍新竹醫院', '和平醫院', '財團法人天主教靈醫會羅東聖母醫院', '財團法人羅許基金會羅東博愛醫院', '衛生福利部苗栗醫院', '財團法人為恭紀念醫院', '臺中榮民總醫院', '衛生福利部豐原醫院', '國軍臺中總醫院附設民眾診療服務處',

### Have at least 22G in your home folder before build docker image 

In [41]:
%%bash 
cd deploy/docker/hubserving/gpu/
docker build . -t paddle --no-cache


Step 1/6 : FROM registry.baidubce.com/paddlepaddle/paddle:2.0.0rc1-gpu-cuda10.0-cudnn7
 ---> 8fcc6640d145
Step 2/6 : RUN pip3.7 install --upgrade pip -i https://mirror.baidu.com/pypi/simple
 ---> Running in 6c065761d8b8
Looking in indexes: https://mirror.baidu.com/pypi/simple
  Found existing installation: pip 18.0
    Uninstalling pip-18.0:
      Successfully uninstalled pip-18.0
Removing intermediate container 6c065761d8b8
 ---> 96a9db984d33
Step 3/6 : RUN pip3.7 install paddlehub --upgrade -i https://mirror.baidu.com/pypi/simple
 ---> Running in 28bbe0d36680
Looking in indexes: https://mirror.baidu.com/pypi/simple
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=2ef378faaa8e25d8931164bbdc323105d52ca2e5c50ae62c92512afbcd35bcf0
  Stored in directory: /root/.cache/pip/wheels/59/7f/60/55701d947bd3c04ab86cdc04064d4814e6c7708cac8920508b
  Created wheel for easydict: filename=easydict-1.9-py3-none-any.whl size=6350 sha256=701789fdc59d921ff2393a9603735

In [43]:
%%bash 
cd ~/SageMaker
mkdir -p model 
nvidia-docker run -v $PWD:/opt/ml/ paddle python3.7 tools/infer_rec.py -c configs/rec/ch_ppocr_v2.0/rec_chinese_lite_train_v2.0.yml -o Global.pretrained_model=pretrain_models/ch_ppocr_mobile_v2.0_rec_pre/best_accuracy Global.load_static_weights=false Global.infer_img=/opt/ml/input/data/setofont_train/train/1020.jpg


[2021/02/25 09:06:38] root INFO: Architecture : 
[2021/02/25 09:06:38] root INFO:     Backbone : 
[2021/02/25 09:06:38] root INFO:         model_name : small
[2021/02/25 09:06:38] root INFO:         name : MobileNetV3
[2021/02/25 09:06:38] root INFO:         scale : 0.5
[2021/02/25 09:06:38] root INFO:         small_stride : [1, 2, 2, 2]
[2021/02/25 09:06:38] root INFO:     Head : 
[2021/02/25 09:06:38] root INFO:         fc_decay : 1e-05
[2021/02/25 09:06:38] root INFO:         name : CTCHead
[2021/02/25 09:06:38] root INFO:     Neck : 
[2021/02/25 09:06:38] root INFO:         encoder_type : rnn
[2021/02/25 09:06:38] root INFO:         hidden_size : 48
[2021/02/25 09:06:38] root INFO:         name : SequenceEncoder
[2021/02/25 09:06:38] root INFO:     Transform : None
[2021/02/25 09:06:38] root INFO:     algorithm : CRNN
[2021/02/25 09:06:38] root INFO:     model_type : rec
[2021/02/25 09:06:38] root INFO: Eval : 
[2021/02/25 09:06:38] root INFO:     dataset : 
[2021/02/25 09:06:38] r

W0225 09:06:38.200100     1 device_context.cc:320] Please NOTE: device: 0, GPU Compute Capability: 3.7, Driver API Version: 11.0, Runtime API Version: 10.0
W0225 09:06:38.204253     1 device_context.cc:330] device: 0, cuDNN Version: 7.6.


In [ ]:
%%bash 

cd ~/SageMaker
mkdir -p model 
nvidia-docker run  -v $PWD:/opt/ml/ paddle python3.7 -m paddle.distributed.launch --gpus '0' tools/train.py -c configs/rec/ch_ppocr_v2.0/rec_chinese_lite_train_seto_v2.0.yml

In [48]:
%%bash 
cd ~/SageMaker
nvidia-docker run -v $PWD:/opt/ml/ paddle python3.7 tools/infer_rec.py -c configs/rec/ch_ppocr_v2.0/rec_chinese_lite_train_seto_v2.0.yml -o Global.pretrained_model=/opt/ml/model/rec_chinese_lite_v2.0/best_accuracy Global.load_static_weights=false Global.infer_img=/opt/ml/input/data/setofont_train/train/1020.jpg

[2021/02/25 12:45:47] root INFO: Architecture : 
[2021/02/25 12:45:47] root INFO:     Backbone : 
[2021/02/25 12:45:47] root INFO:         model_name : small
[2021/02/25 12:45:47] root INFO:         name : MobileNetV3
[2021/02/25 12:45:47] root INFO:         scale : 0.5
[2021/02/25 12:45:47] root INFO:         small_stride : [1, 2, 2, 2]
[2021/02/25 12:45:47] root INFO:     Head : 
[2021/02/25 12:45:47] root INFO:         fc_decay : 1e-05
[2021/02/25 12:45:47] root INFO:         name : CTCHead
[2021/02/25 12:45:47] root INFO:     Neck : 
[2021/02/25 12:45:47] root INFO:         encoder_type : rnn
[2021/02/25 12:45:47] root INFO:         hidden_size : 48
[2021/02/25 12:45:47] root INFO:         name : SequenceEncoder
[2021/02/25 12:45:47] root INFO:     Transform : None
[2021/02/25 12:45:47] root INFO:     algorithm : CRNN
[2021/02/25 12:45:47] root INFO:     model_type : rec
[2021/02/25 12:45:47] root INFO: Eval : 
[2021/02/25 12:45:47] root INFO:     dataset : 
[2021/02/25 12:45:47] r

W0225 12:45:47.156575     1 device_context.cc:320] Please NOTE: device: 0, GPU Compute Capability: 3.7, Driver API Version: 11.0, Runtime API Version: 10.0
W0225 12:45:47.160823     1 device_context.cc:330] device: 0, cuDNN Version: 7.6.
